Make a big pandas dataframe to hold all the metric values we're interested in, so then we can pass it around and plot it up any way we like

I guess depth in each filter for each DDF?



In [1]:
import glob
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pylab as plt
import os
%matplotlib inline

In [2]:
# I just coppied them over from hyak and sym linked them to this directory for ease of use
glance_dirs = glob.glob('./*_glance')
sci_dirs = glob.glob('./*_sci')

glance_dirs.sort()
sci_dirs.sort()


In [3]:
sci_dirs


['./baseline_v2.0_10yrs_sci',
 './minion_1016_update_sci',
 './retro_baseline_v2.0_10yrs_sci']

In [4]:
glance_dirs

['./baseline_v2.0_10yrs_glance',
 './minion_1016_update_glance',
 './retro_baseline_v2.0_10yrs_glance']

In [5]:
filenames = [name.replace('./', '').replace('_glance', '') for name in glance_dirs]

In [6]:
runNames =  [name[2:].replace('_v2.0_10yrs_glance','').replace('v2.0_10yrs_glance','').replace('v2.0_10yrs_glance','').replace('_update_glance', '') for name in glance_dirs] 
versions = ['2.0']*len(glance_dirs) 

In [7]:
runNames

['baseline', 'minion_1016', 'retro_baseline']

In [9]:
runNames = ['Baseline 2.0', 'minion_1016', 'Baseline 1.x']

In [10]:
# Set up the dataframe
df = pd.DataFrame(np.array([runNames, versions]).T, columns=['runName', 'version'])

In [11]:
# List of tupes with (metricName, summaryName) that we want to pull from glance dirs
mnamesname =[('parallax', 'best18k'),
            ('properMotion', 'best18k'),
            ('fO', 'fONv MedianNvis'),
            ('fO', 'fOArea')]

In [12]:
for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s") and summaryName= "%s";' % (names[0], names[1])
    for directory in glance_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

In [13]:
# Add the median coadded depths in each filter
for filtername in 'ugrizy':
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "CoaddM5" and metricMetadata=" %s") and summaryName= "Median";' % (filtername)
    for directory in glance_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df['median_coadd_%s' % filtername] = column_vals


In [14]:
# List of tupes with (metricName, summaryName) that we want to pull from science dirs
mnamesname =[('SNNSNMetric_nSN', 'Total detected'),
            ('1 to 10 day Microlensing', 'Fraction detected of total (mean)'),
            ('100 to 1000 day Microlensing', 'Fraction detected of total (mean)'),
            ('KNePopMetric__multi_color_detect', 'Fraction detected of total (mean)'),
            ('GalaxyCountsMetric_extended', 'N Galaxies (all)'),
            ('TDEsPopMetric__some_color', 'Fraction detected of total (mean)'),
             ('TDEsPopMetric__some_color_pu', 'Fraction detected of total (mean)'), 
            ('WeakLensingNvisits', 'Median')]

In [15]:
for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

In [16]:
sql

'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "WeakLensingNvisits") and summaryName= "Median";'

In [17]:
# things that need some metadata
mnamesname =[('Nstars_no_crowding', 'Total N Stars, no crowding')]

for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s" and metricMetadata=" i") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

    
mnamesname =[('ExgalM5_with_cuts', '3x2ptFoM')]

for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s" and metricMetadata="i band non-DD year 10") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            try:
                column_vals.append(np.max(result))
            except:
                import pdb ;pdb.set_trace()
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals


In [18]:
np.max(result)

34.6258906699115

In [19]:
# Pull the DDF depths from everywhere. 
filters = 'ugrizy'
ddf_names = ['DD:ELAISS1', 'DD:XMM-LSS', 'DD:ECDFS', 'DD:COSMOS', 'DD:EDFSa', 'DD:EDFSb']

for filtername in filters:
    for ddf_name in ddf_names:
        column_vals = []
        sql = 'select summaryValue from summarystats where summaryName="Median depth %s, %s"' % (ddf_name, filtername)
        for directory in sci_dirs:
            conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
            result = pd.read_sql(sql, conn).values.ravel()
            if np.size(result) > 1:
                ValueError
            else:
                column_vals.append(np.max(result))
            conn.close()
        df[ddf_name+','+filtername] = column_vals

In [20]:
sql

'select summaryValue from summarystats where summaryName="Median depth DD:EDFSb, y"'

In [21]:
# Now to pull things from the solar system results

# here's what we used last time:
#'3 pairs in 15 nights detection loss NEO H=16.0',
#        '3 pairs in 30 nights detection loss NEO H=22.0',
#       '3 pairs in 15 nights detection loss TNO H=4.0'

faint_neo = []
bright_neo = []
tno = []

for name in filenames:
    conn = sqlite3.connect('%s_ss/resultsDb_sqlite.db' % name)
    sql = 'select summaryValue from summarystats where summaryName="DifferentialCompleteness H = 16.000000" and '+\
        'metricId = (select metricId from metrics where metricName ="Discovery_N_Chances" and metricMetadata="NEO 3 pairs in 15 nights detection loss")'
    val = np.max(pd.read_sql(sql, conn).values.ravel())
    bright_neo.append(val)
    
    
    sql = 'select summaryValue from summarystats where summaryName="DifferentialCompleteness H = 22.000000" and '+\
        'metricId = (select metricId from metrics where metricName ="Discovery_N_Chances" and metricMetadata="NEO 3 pairs in 15 nights detection loss")'

    val = np.max(pd.read_sql(sql, conn).values.ravel())
    faint_neo.append(val)
    
    #conn.close()
    #conn = sqlite3.connect('%s_l7_5k/resultsDb_sqlite.db' % name)
    
    sql = 'select summaryValue from summarystats where summaryName="DifferentialCompleteness H = 6.000000" and '+\
          'metricId = (select metricId from metrics where metricName ="Discovery_N_Chances" and metricMetadata="TNO 3 pairs in 15 nights detection loss")'

    val = np.max(pd.read_sql(sql, conn).values.ravel())
    tno.append(val)
    conn.close()
    
df['NEO bright'] = bright_neo
df['NEO faint'] = faint_neo 
df['TNO'] = tno

In [22]:
'%s_ss/resultsDb_sqlite.db' % name

'retro_baseline_v2.0_10yrs_ss/resultsDb_sqlite.db'

In [23]:
df['NEO faint']

0    0.3652
1    0.4026
2    0.3690
Name: NEO faint, dtype: float64

In [24]:
df['NEO bright']

0    0.9296
1    0.9300
2    0.9094
Name: NEO bright, dtype: float64

In [25]:
df['TNO']

0    0.6480
1    0.6364
2    0.6364
Name: TNO, dtype: float64

In [26]:
runNames[0]

'Baseline 2.0'

In [ ]:
plt.plot(df['median_coadd_g'])

In [ ]:
df.columns

In [28]:
# pickle would be much smaller, but I guess can zip after if we want to
df.to_hdf('combined_maf_dataframe.hdf', 'maf')
# restore with:
# df = pd.read_hdf('combined_maf_dataframe.hdf', 'maf')

In [27]:
df

,runName,version,parallax_best18k,properMotion_best18k,fO_fONv MedianNvis,fO_fOArea,median_coadd_u,median_coadd_g,median_coadd_r,median_coadd_i,...,"DD:EDFSb,z","DD:ELAISS1,y","DD:XMM-LSS,y","DD:ECDFS,y","DD:COSMOS,y","DD:EDFSa,y","DD:EDFSb,y",NEO bright,NEO faint,TNO
0,Baseline 2.0,2.0,0.791058,0.184088,839.0,12893.228978,25.818916,26.733479,26.755971,26.218659,...,26.724914,26.727387,26.621554,26.667555,26.604660,26.468794,26.494803,0.9296,0.3652,0.6480
1,minion_1016,2.0,0.844273,0.224752,770.0,7745.001758,25.575596,26.711079,26.700751,26.226561,...,25.550946,24.593499,26.311665,26.653766,26.395957,24.718278,24.740604,0.9300,0.4026,0.6364
2,Baseline 1.x,2.0,0.804328,0.176701,848.0,14611.263069,25.707301,26.835006,26.850968,26.308518,...,26.881648,26.716894,26.617415,26.587666,26.583537,26.478199,26.473841,0.9094,0.3690,0.6364
